## Topic Modeling of Lyrics with LDA

Author: Miles Mezaki

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('data/labeled_lyrics_cleaned.csv')
df

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371
...,...,...,...,...,...
158348,158348,Adam Green,"And we live on borrowed time,\r\nBut this head...",Friends of Mine,0.737
158349,158349,Adam Green,Frozin in time forever\r\nCarrying that torch ...,Frozen in Time,0.482
158350,158350,Adam Green,Hard to be a girl. \r\nSo nice to be a boy. \r...,Hard to Be a Girl,0.733
158351,158351,Adam Green,"I want to chose to die,\r\nAnd be buried with ...",I Wanna Die,0.361


In [6]:
add_on = pd.read_csv('songs_with_lyrics.csv')
add_on['song'] = add_on['track_name']
add_on['seq'] = add_on['lyrics']
add_on

,file_name,track_name,artist,lyrics,song,seq
0,share_video_7329274198096973099_.mp3,Montagem Mysterious Game,LXNGVX,This song is an instrumental,Montagem Mysterious Game,This song is an instrumental
1,share_video_7339685917759769899_.mp3,Dumptruck,Kinfolk Thugs,[Intro]\nAh-where they at?\n(K-K-K-Kin Folk Th...,Dumptruck,[Intro]\nAh-where they at?\n(K-K-K-Kin Folk Th...
2,share_video_7324083003284573482_.mp3,Little Life,Cordelia,[Verse 1]\nHow would you have me described?\nW...,Little Life,[Verse 1]\nHow would you have me described?\nW...
3,share_video_7303373812387679490_.mp3,A Sky Full of Stars (Live at the Royal Albert ...,Coldplay,"[Intro]\nOh, turn the lights on, turn the ligh...",A Sky Full of Stars (Live at the Royal Albert ...,"[Intro]\nOh, turn the lights on, turn the ligh..."
4,share_video_7306351228974288159_.mp3,How To Never Stop Being Sad,dandelion hands,Repeat to yourself that they're not really gon...,How To Never Stop Being Sad,Repeat to yourself that they're not really gon...
...,...,...,...,...,...,...
363,NaN,In The Party,Flo Milli,La la la la la la Flo Milli shit La la la la l...,In The Party,La la la la la la Flo Milli shit La la la la l...
364,NaN,Beef FloMix,Flo Milli,Hah Flo Milli shit bitch I like cash and my ...,Beef FloMix,Hah Flo Milli shit bitch I like cash and my ...
365,NaN,Rodeo (Remix),Flo Milli,Yeah I wanna fuck you right now Reverse that...,Rodeo (Remix),Yeah I wanna fuck you right now Reverse that...
366,NaN,Conceited,Flo Milli,Thats Fyre Feelin myself Im conceited Huh Fe...,Conceited,Thats Fyre Feelin myself Im conceited Huh Fe...


In [7]:
merged_df = pd.merge(df, add_on, on=['artist','seq','song'], how='left')
merged_df
seen = set()
to_drop = []
for i,row in df.iterrows():
    if (row['artist'], row['song']) in seen:
        to_drop.append(i)
    seen.add((row['artist'], row['song']))

In [8]:
print(len(merged_df))
df = merged_df.drop(to_drop)
print(len(df))

158353
158312


In [50]:
to_drop

[9970,
 9971,
 9976,
 42862,
 43183,
 45717,
 45718,
 45719,
 50648,
 57111,
 57119,
 57150,
 74966,
 74967,
 75636,
 78347,
 83229,
 94212,
 96443,
 96445,
 96446,
 96447,
 108268,
 120201,
 125247,
 125277,
 125278,
 125280,
 125281,
 125282,
 125287,
 125288,
 125289,
 125290,
 125291,
 125292,
 125296,
 133164,
 141742,
 147526,
 155274]

In [53]:
df.iloc[9965:9975]

,Unnamed: 0,artist,seq,song,label,file_name,track_name,lyrics,terms
9965,9965,Andy Williams,"moon river, wider than a mile i'm crossing you...","Moon River (From ""Breakfast at Tiffany's"")",0.1950,NaN,NaN,NaN,"[bend, breaker, crossing, day, dream, drifters..."
9966,9966,Andy Williams,night and you and blue hawaii the night is hea...,Blue Hawaii,0.6460,NaN,NaN,NaN,"[blue, come, dreams, hawaii, heaven, heavenly,..."
9967,9967,Andy Williams,i found my thrill on blueberry hill on blueber...,Blueberry Hill,0.5810,NaN,NaN,NaN,"[apart, blueberry, came, dreams, hill, lingere..."
9968,9968,Andy Williams,"you tell me you love me, you say youll be true...",Butterfly,0.8830,NaN,NaN,NaN,"[butterfly, clipping, crazy, drips, fly, flyin..."
9969,9969,Andy Williams,hes got the whole world in his hands hes got t...,He's Got the Whole World in His Hands,0.3860,NaN,NaN,NaN,"[baby, bitty, brother, everybody, got, hands, ..."
9972,9972,Andy Williams,on a day like today we pass the time away writ...,Love Letters in the Sand,0.5950,NaN,NaN,NaN,"[aches, away, breaks, broken, cried, day, hear..."
9973,9973,Andy Williams,this is the night of love the shining hour of ...,Love Song of Kalua,0.3670,NaN,NaN,NaN,"[air, arms, blossom, blow, bright, hold, hour,..."
9974,9974,Andy Williams,evening shadows make me blue when each weary d...,My Happiness,0.6490,NaN,NaN,NaN,"[blue, day, dreaming, dreams, earth, evening, ..."
9975,9975,Andy Williams,theres just one place for me near you its like...,Near You,0.3860,NaN,NaN,NaN,"[apart, away, content, days, face, happy, hear..."
9977,9977,Andy Williams,"some enchanted evening, you may see a stranger...",Some Enchanted Evening,0.0686,NaN,NaN,NaN,"[crowded, dream, dreams, enchanted, evening, e..."


In [57]:
df = df.reset_index(drop=True)
df.iloc[9965:9975]

,Unnamed: 0,artist,seq,song,label,file_name,track_name,lyrics,terms
9965,9965,Andy Williams,"moon river, wider than a mile i'm crossing you...","Moon River (From ""Breakfast at Tiffany's"")",0.1950,NaN,NaN,NaN,"[bend, breaker, crossing, day, dream, drifters..."
9966,9966,Andy Williams,night and you and blue hawaii the night is hea...,Blue Hawaii,0.6460,NaN,NaN,NaN,"[blue, come, dreams, hawaii, heaven, heavenly,..."
9967,9967,Andy Williams,i found my thrill on blueberry hill on blueber...,Blueberry Hill,0.5810,NaN,NaN,NaN,"[apart, blueberry, came, dreams, hill, lingere..."
9968,9968,Andy Williams,"you tell me you love me, you say youll be true...",Butterfly,0.8830,NaN,NaN,NaN,"[butterfly, clipping, crazy, drips, fly, flyin..."
9969,9969,Andy Williams,hes got the whole world in his hands hes got t...,He's Got the Whole World in His Hands,0.3860,NaN,NaN,NaN,"[baby, bitty, brother, everybody, got, hands, ..."
9970,9972,Andy Williams,on a day like today we pass the time away writ...,Love Letters in the Sand,0.5950,NaN,NaN,NaN,"[aches, away, breaks, broken, cried, day, hear..."
9971,9973,Andy Williams,this is the night of love the shining hour of ...,Love Song of Kalua,0.3670,NaN,NaN,NaN,"[air, arms, blossom, blow, bright, hold, hour,..."
9972,9974,Andy Williams,evening shadows make me blue when each weary d...,My Happiness,0.6490,NaN,NaN,NaN,"[blue, day, dreaming, dreams, earth, evening, ..."
9973,9975,Andy Williams,theres just one place for me near you its like...,Near You,0.3860,NaN,NaN,NaN,"[apart, away, content, days, face, happy, hear..."
9974,9977,Andy Williams,"some enchanted evening, you may see a stranger...",Some Enchanted Evening,0.0686,NaN,NaN,NaN,"[crowded, dream, dreams, enchanted, evening, e..."


Define code to expand contracted words, which are prominent in music:

In [58]:
def expand_contractions(s):
    res = ""
    contractions = { 
        "ain't": "aint", # Could be many tenses, but is not is probably the best mapping for all tenses
        "aren't": "arent",
        "can't": "cant",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "couldve",
        "couldn't": "couldnt",
        "couldn't've": "couldnt have",
        "didn't": "didnt",
        "doesn't": "doesnt",
        "don't": "dont",
        "hadn't": "hadnt",
        "hadn't've": "had not have",
        "hasn't": "hasnt",
        "haven't": "havent",
        "he'd": "hed",
        "he'd've": "he wouldve",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "hes",
        "how'd": "howd",
        "how'd'y": "howd you",
        "how'll": "how will",
        "how's": "hows",
        "I'd": "Id",
        "I'd've": "I wouldve",
        "I'll": "I will",
        "I'll've": "I will have",
        "I'm": "Im",
        "I've": "Ive",
        "isn't": "isnt",
        "it'd": "it would",
        "it'd've": "it wouldve",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "its",
        "let's": "lets",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "mightve",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "mustve",
        "musta" : "mustve",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "shed",
        "she'd've": "she would have",
        "she'll": "shell",
        "she'll've": "she will have",
        "she's": "shes",
        "should've": "shouldve",
        "shouldn't": "shouldnt",
        "shouldn't've": "shouldnt have",
        "so've": "sove",
        "so's": "sos",
        "that'd": "thatd",
        "that'd've": "thatd have",
        "that's": "thats",
        "there'd": "thered",
        "there'd've": "thered have",
        "there's": "theres",
        "they'd": "theyd",
        "they'd've": "theyd have",
        "they'll": "theyll",
        "they'll've": "theyll have",
        "they're": "theyre",
        "they've": "theyve",
        "to've": "to have",
        "wasn't": "wasnt",
        "we'd": "wed",
        "we'd've": "we wouldve",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "weve",
        "weren't": "werent",
        "what'll": "whatll",
        "what'll've": "whatll have",
        "what're": "whatre",
        "what's": "whats",
        "what've": "whatve",
        "when's": "whens",
        "when've": "whenve",
        "where'd": "whered",
        "where's": "wheres",
        "where've": "whereve",
        "who'll": "wholl",
        "who'll've": "wholl have",
        "who's": "whos",
        "who've": "whove",
        "why's": "whys",
        "why've": "whyve",
        "will've": "will have",
        "won't": "wont",
        "won't've": "wont have",
        "would've": "wouldve",
        "wouldn't": "wouldnt",
        "wouldn't've": "wouldnt have",
        "y'all": "yall",
        "y'all'd": "yall would",
        "y'all'd've": "yall would have",
        "y'all're": "yall are",
        "y'all've": "yall have",
        "you'd": "youd",
        "you'd've": "youd have",
        "you'll": "youll",
        "you'll've": "youll have",
        "you're": "youre",
        "you've": "youve"
        }
    for word in s.split():
        if word in contractions:
            res += contractions[word] + " "
        else:
            res += word + " "
    return res

print(df['seq'][0])

no, no i aint ever trapped out the bando but oh lord, dont get me wrong i know a couple niggas that do i'm from a place where everybody knows your name they say i gotta watch my attitude when they see money, man they all start acting strange so fuck with the ones that fuck with you they can never say i'm brand new its everyday, everyday everyday, everyday, everyday everyday, everyday everyday, everyday i've been talking my shit, nigga thats regular i've been loving 'em thick, life is spectacular i spend like i'ma die rich, nigga i'm flexing, yeah everyday, thats everyday thats everyday thats everyday thats everyday, everyday i see all of these wanna-be hot r&b singers i swear you all sound the same they start from the bottom, so far from the motto you niggas'll never be drake shout out to ovo most of them prolly dont know me though i stay in the cut, i dont fuck with no body but i d, thats a pun on no i.d when nobody know my name running for my dream wasnt hard to do you break bread, i

In [59]:
expand_contractions(df['seq'][0])

"no, no i aint ever trapped out the bando but oh lord, dont get me wrong i know a couple niggas that do i'm from a place where everybody knows your name they say i gotta watch my attitude when they see money, man they all start acting strange so fuck with the ones that fuck with you they can never say i'm brand new its everyday, everyday everyday, everyday, everyday everyday, everyday everyday, everyday i've been talking my shit, nigga thats regular i've been loving 'em thick, life is spectacular i spend like i'ma die rich, nigga i'm flexing, yeah everyday, thats everyday thats everyday thats everyday thats everyday, everyday i see all of these wanna-be hot r&b singers i swear you all sound the same they start from the bottom, so far from the motto you niggas'll never be drake shout out to ovo most of them prolly dont know me though i stay in the cut, i dont fuck with no body but i d, thats a pun on no i.d when nobody know my name running for my dream wasnt hard to do you break bread, 

Next define code that will match expressions missing the trailing g in their progressive tense and replace the apostrophe with a g. (e.g. trippin' -> tripping)

In [60]:
import re

pattern = r"(.)in'"
replacement =  r"\1ing"
def expand_verbs(s: str, pattern, replacement):
    """
    Assume s is the lyrics string
    """
    res, _ = re.subn(pattern, replacement, s)
    return res

expand_verbs("trippin', grippin', dribblin'.", pattern, replacement)

'tripping, gripping, dribbling.'

In [61]:
def expand_verbs_and_contractions(s, pattern=r"(.)in'", replacement=r"\1ing"):
    """
    s: string representing lyrics
    pattern: regular expression representing the pattern to be replaced
    replacement: regular expression to replace pattern

    Returns:
        res: string ready for tokenization
    """
    return expand_verbs(expand_contractions(s.lower()), pattern, replacement)
    
expand_verbs_and_contractions(df['seq'][0])

"no, no i aint ever trapped out the bando but oh lord, dont get me wrong i know a couple niggas that do i'm from a place where everybody knows your name they say i gotta watch my attitude when they see money, man they all start acting strange so fuck with the ones that fuck with you they can never say i'm brand new its everyday, everyday everyday, everyday, everyday everyday, everyday everyday, everyday i've been talking my shit, nigga thats regular i've been loving 'em thick, life is spectacular i spend like i'ma die rich, nigga i'm flexing, yeah everyday, thats everyday thats everyday thats everyday thats everyday, everyday i see all of these wanna-be hot r&b singers i swear you all sound the same they start from the bottom, so far from the motto you niggas'll never be drake shout out to ovo most of them prolly dont know me though i stay in the cut, i dont fuck with no body but i d, thats a pun on no i.d when nobody know my name running for my dream wasnt hard to do you break bread, 

In [62]:
df['seq'] = df['seq'].apply(expand_verbs_and_contractions)

In [63]:
df.head()

,Unnamed: 0,artist,seq,song,label,file_name,track_name,lyrics,terms
0,0,Elijah Blake,"no, no i aint ever trapped out the bando but o...",Everyday,0.626,NaN,NaN,NaN,"[acting, aint, attitude, bando, body, brand, b..."
1,1,Elijah Blake,"the drinks go down and smoke goes up, i feel m...",Live Till We Die,0.630,NaN,NaN,NaN,"[ace, aint, away, band, bite, blow, cares, cha..."
2,2,Elijah Blake,she dont live on planet earth no more she foun...,The Otherside,0.240,NaN,NaN,NaN,"[bad, bags, broke, broken, called, calling, ca..."
3,3,Elijah Blake,"tripping off that grigio, mobbing, lights low ...",Pinot,0.536,NaN,NaN,NaN,"[aint, baby, beginnings, blow, boy, calling, c..."
4,4,Elijah Blake,"i see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371,NaN,NaN,NaN,"[answers, believe, brave, broke, coal, coffee,..."


Next, we want to tokenize and stem the data. We will not remove stop words since the people are generally important in lyrics and the verbs are often stopwords.

In [64]:
# from nltk.tokenize import word_tokenize 

# text = df['seq'][0]
# # tokenize
# result = word_tokenize(text)
# result

In [65]:
# from nltk.stem import PorterStemmer
 
# ps = PorterStemmer()
 
# # choose some words to be stemmed
# words = df['seq'][3]
 
# for w in words.split():
#     print(w, " : ", ps.stem(w))

In [66]:
# def token_stem(s):
#     """
#     Given a string s, tokenize the string, then stem it.
#     """
#     ps = PorterStemmer()
#     word_tokenize(s)
#     return " ".join([ps.stem(w) for w in word_tokenize(s)])

# token_stem("converts the words in word_tokens to lower case and then checks whether ")

Now we apply the tokenization and stemming to our lyrics.

In [67]:
# df['seq'] = df['seq'].apply(token_stem)

Now our lyrics text is ready for post-processing.

In [68]:
df['seq']

0         no, no i aint ever trapped out the bando but o...
1         the drinks go down and smoke goes up, i feel m...
2         she dont live on planet earth no more she foun...
3         tripping off that grigio, mobbing, lights low ...
4         i see a midnight panther, so gallant and so br...
                                ...                        
158307    and we live on borrowed time, but this headsho...
158308    frozin in time forever carrying that torch for...
158309    hard to be a girl. so nice to be a boy. in my ...
158310    i want to chose to die, and be buried with a r...
158311    musical ladders leaning on mountains bathed in...
Name: seq, Length: 158312, dtype: object

In [69]:
from sklearn.feature_extraction.text import CountVectorizer

In [70]:
# Initialize the vectorizer
vectorizer = CountVectorizer(
    strip_accents='unicode',
    stop_words='english',
    lowercase=True,
    token_pattern=r'\b[a-zA-Z]{3,}\b', # we want only words that contain letters and are 3 or more characters long
)

# Transform our data into the document-term matrix
dtm = vectorizer.fit_transform(df['seq'])
dtm

<158312x132479 sparse matrix of type '<class 'numpy.int64'>'
	with 8422154 stored elements in Compressed Sparse Row format>

In [71]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['aaa', 'aaaa', 'aaaaa', ..., 'zzzz', 'zzzzs', 'zzzzzombieee'],
      dtype=object)

In [72]:
feature_names.shape

(132479,)

In [73]:
feature_names[300:350]

array(['abductee', 'abductees', 'abducting', 'abduction', 'abductor',
       'abducts', 'abdul', 'abdullah', 'abe', 'abeam', 'abebe',
       'abecedario', 'abed', 'abednago', 'abednego', 'abeds', 'abeg',
       'abeille', 'abeja', 'abejita', 'abel', 'abelard', 'abelene',
       'abeline', 'abell', 'abend', 'abende', 'abendigo', 'abends',
       'abenteuer', 'aber', 'abercrombie', 'aberdeen', 'abernathy',
       'aberrant', 'aberration', 'aberrations', 'aberrettes', 'aberta',
       'abetter', 'abetting', 'abeyance', 'abgefahren', 'abgefickt',
       'abgehn', 'abgesaugt', 'abgestellt', 'abgrund', 'abgvll', 'abh'],
      dtype=object)

In [74]:
doc1 = dtm[0]
doc1

<1x132479 sparse matrix of type '<class 'numpy.int64'>'
	with 78 stored elements in Compressed Sparse Row format>

In [75]:
row_index = 0
doc_vec = dtm.getrow(row_index).toarray()

non_zero_indices = doc_vec.nonzero()[1]
dtm_scores = doc_vec[0, non_zero_indices] # goes and retrieves the values corresponding to the non_zero_indices
words = [feature_names[i] for i in non_zero_indices]

for word, score in zip(words, dtm_scores):
    print(f"{word}: {score}")

acting: 2
aint: 2
attitude: 1
bando: 1
body: 1
brand: 1
bread: 1
break: 1
change: 1
changed: 1
couple: 1
crew: 1
cut: 1
die: 3
dont: 3
drake: 1
dream: 1
dress: 1
eat: 1
everybody: 1
everyday: 45
far: 1
flexing: 3
fuck: 3
funny: 1
gets: 1
got: 2
gotta: 1
hard: 1
heard: 1
hobby: 1
hot: 1
know: 3
knows: 1
life: 3
like: 3
lonely: 1
lord: 1
lose: 1
loving: 4
man: 1
money: 2
motto: 1
new: 1
nigga: 6
niggas: 2
ones: 2
ovo: 1
place: 1
plate: 1
prolly: 1
pull: 1
pun: 1
regular: 3
rich: 3
running: 1
say: 2
shit: 3
shout: 1
singers: 1
sound: 1
spectacular: 3
spend: 3
start: 2
starved: 1
stay: 1
strange: 1
swear: 2
talking: 3
thats: 17
trapped: 1
wanna: 1
wasnt: 1
watch: 1
winning: 1
wrong: 1
yeah: 3
youre: 1


In [76]:
non_zero_indices

array([  1069,   2333,   6707,   8438,  12828,  14106,  14263,  14282,
        19060,  19063,  25166,  25817,  26988,  30896,  33062,  33698,
        33812,  33909,  35121,  38557,  38563,  40120,  42249,  44393,
        44724,  46307,  47993,  48019,  50766,  51558,  53284,  54211,
        63166,  63210,  66278,  66435,  67487,  67716,  67768,  67986,
        69649,  74723,  75576,  78254,  78575,  78595,  81252,  82951,
        87339,  87521,  90449,  91236,  91321,  94639,  96536,  98625,
       100223, 103866, 104237, 105162, 108230, 108691, 108783, 110185,
       110217, 110319, 111230, 113605, 114512, 116382, 119423, 127116,
       127358, 127434, 129353, 130402, 130969, 131580])

In [77]:
dtm.getcol(2327).toarray().T # get the column, turn it into an array format, then transpose it to be a row

array([[0, 0, 0, ..., 0, 0, 0]])

In [78]:
dtm.getcol(44592).toarray().T # Fuck, which should appear often in songs

array([[0, 0, 0, ..., 0, 0, 0]])

In [79]:
np.count_nonzero(dtm.getcol(44592).toarray().T)

1

Back to DF

In [80]:
def matrix2Doc(dtMatrix, features, index):
    """Turns each row of the document-term matrix into a list of terms"""
    row = dtMatrix.getrow(index).toarray()
    non_zero_indices = row.nonzero()[1]
    words = [features[idx] for idx in non_zero_indices]
    return words

In [81]:
allDocsAsTerms = [matrix2Doc(dtm, feature_names, i) for i in range(dtm.shape[0])]

In [82]:
len(allDocsAsTerms)

158312

In [83]:
df['terms'] = allDocsAsTerms
df.head()

,Unnamed: 0,artist,seq,song,label,file_name,track_name,lyrics,terms
0,0,Elijah Blake,"no, no i aint ever trapped out the bando but o...",Everyday,0.626,NaN,NaN,NaN,"[acting, aint, attitude, bando, body, brand, b..."
1,1,Elijah Blake,"the drinks go down and smoke goes up, i feel m...",Live Till We Die,0.630,NaN,NaN,NaN,"[ace, aint, away, band, bite, blow, cares, cha..."
2,2,Elijah Blake,she dont live on planet earth no more she foun...,The Otherside,0.240,NaN,NaN,NaN,"[bad, bags, broke, broken, called, calling, ca..."
3,3,Elijah Blake,"tripping off that grigio, mobbing, lights low ...",Pinot,0.536,NaN,NaN,NaN,"[aint, baby, beginnings, blow, boy, calling, c..."
4,4,Elijah Blake,"i see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371,NaN,NaN,NaN,"[answers, believe, brave, broke, coal, coffee,..."


In [84]:
from sklearn.decomposition import LatentDirichletAllocation

# Step 1: Initialize the model

lda = LatentDirichletAllocation(n_components=5, # based on GridSearch, we discover that 5 is a good number of topics for LDA.
                                random_state=0)

# Step 2: Fit the model
lda.fit(dtm)

LatentDirichletAllocation(n_components=5, random_state=0)

In [85]:
lda.components_

array([[ 1.91407257,  0.20670357,  0.20015286, ...,  0.20191532,
         0.20131285,  0.20000034],
       [ 0.20157707,  1.53931829,  0.20059938, ...,  0.20000324,
         0.20040976,  0.20111699],
       [32.40852736,  0.20733811,  4.19793504, ...,  1.06573406,
         0.20000036,  0.2044806 ],
       [ 0.42104707,  0.20388223,  0.2000004 , ...,  0.20000269,
         0.20000024,  0.20269935],
       [13.05477593,  5.8427578 ,  0.20131232, ...,  0.33234468,
         1.19827678,  1.19170271]])

In [86]:
lda.components_.shape

(5, 132479)

In [87]:
def display_topics(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([features[i]
                        for i in topic.argsort()[:-no_top_words-1:-1]])) # syntax for reversing a list [::-1]

display_topics(lda, feature_names, 10)

Topic 0:
dont know just love youre baby want like got time
Topic 1:
god world lord man like life light blood black dead
Topic 2:
yeah come time gonna night home let just day away
Topic 3:
love heart like life eyes day away night just dream
Topic 4:
like got aint dont man know just nigga shit money


In [88]:
doc_topic_dist = lda.transform(dtm)
doc_topic_dist 

array([[0.42419656, 0.00110307, 0.00110362, 0.00111064, 0.5724861 ],
       [0.10766649, 0.0008916 , 0.21419202, 0.46934548, 0.20790441],
       [0.5131472 , 0.00167564, 0.00166322, 0.00166639, 0.48184755],
       ...,
       [0.40129839, 0.00550097, 0.14938615, 0.00541719, 0.4383973 ],
       [0.09491181, 0.42723951, 0.00386795, 0.0038652 , 0.47011554],
       [0.0038055 , 0.21503481, 0.29580831, 0.20815261, 0.27719876]])

In [89]:
doc_topic_dist.shape

(158312, 5)

In [90]:
def displayHeader(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    topicNames = []
    for topic_idx, topic in enumerate(model.components_):
        topicNames.append(f"Topic {topic_idx}: " + (", ".join([features[i]
                             for i in topic.argsort()[:-no_top_words-1:-1]])))
    return topicNames

In [91]:
# column names
topicnames = displayHeader(lda, feature_names, 10)

# index names
docnames = df.index.tolist() # We will use the original names of the documents

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(doc_topic_dist, 3), 
                                 columns=topicnames, 
                                 index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1) # finds the maximum argument
df_document_topic['dominant_topic'] = dominant_topic

df_document_topic.head()

,"Topic 0: dont, know, just, love, youre, baby, want, like, got, time","Topic 1: god, world, lord, man, like, life, light, blood, black, dead","Topic 2: yeah, come, time, gonna, night, home, let, just, day, away","Topic 3: love, heart, like, life, eyes, day, away, night, just, dream","Topic 4: like, got, aint, dont, man, know, just, nigga, shit, money",dominant_topic
0,0.424,0.001,0.001,0.001,0.572,4
1,0.108,0.001,0.214,0.469,0.208,3
2,0.513,0.002,0.002,0.002,0.482,0
3,0.290,0.001,0.063,0.001,0.645,4
4,0.003,0.003,0.079,0.642,0.273,3


In [92]:
df_document_topic.iloc[9970]

Topic 0: dont, know, just, love, youre, baby, want, like, got, time      0.005
Topic 1: god, world, lord, man, like, life, light, blood, black, dead    0.005
Topic 2: yeah, come, time, gonna, night, home, let, just, day, away      0.005
Topic 3: love, heart, like, life, eyes, day, away, night, just, dream    0.979
Topic 4: like, got, aint, dont, man, know, just, nigga, shit, money      0.005
dominant_topic                                                           3.000
Name: 9970, dtype: float64

In [93]:
df_document_topic['song'] = df['song']
df_document_topic['artist'] = df['artist']
df_document_topic

,"Topic 0: dont, know, just, love, youre, baby, want, like, got, time","Topic 1: god, world, lord, man, like, life, light, blood, black, dead","Topic 2: yeah, come, time, gonna, night, home, let, just, day, away","Topic 3: love, heart, like, life, eyes, day, away, night, just, dream","Topic 4: like, got, aint, dont, man, know, just, nigga, shit, money",dominant_topic,song,artist
0,0.424,0.001,0.001,0.001,0.572,4,Everyday,Elijah Blake
1,0.108,0.001,0.214,0.469,0.208,3,Live Till We Die,Elijah Blake
2,0.513,0.002,0.002,0.002,0.482,0,The Otherside,Elijah Blake
3,0.290,0.001,0.063,0.001,0.645,4,Pinot,Elijah Blake
4,0.003,0.003,0.079,0.642,0.273,3,Shadows & Diamonds,Elijah Blake
...,...,...,...,...,...,...,...,...
158307,0.655,0.003,0.003,0.152,0.187,0,Friends of Mine,Adam Green
158308,0.469,0.365,0.004,0.159,0.004,0,Frozen in Time,Adam Green
158309,0.401,0.006,0.149,0.005,0.438,4,Hard to Be a Girl,Adam Green
158310,0.095,0.427,0.004,0.004,0.470,4,I Wanna Die,Adam Green


In [94]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,64371
1,3,34116
2,1,25053
3,4,19504
4,2,15268


In [95]:
df_document_topic.to_csv('../lda-results/document_topic_with_lyrics_and_top_10.csv')
df_topic_distribution.to_csv('../lda-results/topic_distribution_with_lyrics_and_top_10.csv')

In [37]:
from sklearn.model_selection import GridSearchCV

# We are going to test multiple values for the number of topics
search_params = {'n_components': [3, 4, 5]}

# Initialize the LDA model
lda = LatentDirichletAllocation()

# Initialize a Grid Search with cross-validation instance
grid = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
grid.fit(dtm)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'n_components': [3, 4, 5]})

In [38]:
grid.cv_results_

{'mean_fit_time': array([322.20832949, 342.09373679, 344.99867043]),
 'std_fit_time': array([22.70609384, 15.30608217,  4.5620314 ]),
 'mean_score_time': array([5.43091664, 6.09414873, 6.06560345]),
 'std_score_time': array([0.43705359, 0.36575259, 0.13991072]),
 'param_n_components': masked_array(data=[3, 4, 5],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_components': 3}, {'n_components': 4}, {'n_components': 5}],
 'split0_test_score': array([-23046081.20111998, -23050772.35207721, -23012318.59184919]),
 'split1_test_score': array([-23663310.51873978, -23555950.55534031, -23633661.00556118]),
 'split2_test_score': array([-23765093.48499016, -23865413.89566913, -23782554.40758479]),
 'split3_test_score': array([-24209871.31897045, -24168308.25315203, -24168346.24125382]),
 'split4_test_score': array([-23689125.6136878 , -23637105.46491072, -23640879.92009572]),
 'mean_test_score': array([-23674696.42750163, -23655510.104

In [39]:
# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

Best Model's Params:  {'n_components': 5}
Best Log Likelihood Score:  -23647552.03326894
Model Perplexity:  1956.6398390262623
